# 소셜 데이터 분석을 위한 크롤링

비, 흐림, 안개와 같은 좋지 않은 날씨의 해시태그들과 부정적인 감정을 표현한 해시태그가 많이 사용되는지 알아보기 위함. 상관관계라도 찾을 수 있으면 더할 나위 없이 좋겠다.

### 첫 번째 시도
참고: [링크 텍스트](https://mokeya.tistory.com/68)

In [185]:
pip install selenium

In [186]:
pip install lxml

In [187]:
#필요 패키지 호출 
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import time

In [188]:
#출력 한 번에 나올 수 있게 설정
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [189]:
#함수 정의: 검색어 조건에 따른 url 생성
# 내가 원하는 키워드를 받아서 해당 키워드 검색 결과 페이지의 url을 생성하는 함수를 정의
def insta_searching(word):
  url="https://www.instagram.com/explore/tags/" +str(word)
  return url

In [190]:
import time 
def select_first(driver):
  first = driver.find_elements_by_css_selector("div._9AhH0")[0]
  first.click()
  time.sleep(3) #로딩을 위해 3초 대기

In [191]:
#함수 정의: 본문 내용, 작성일자, 좋아요 수, 위치정보, 해시태그 가져오기
import re
def get_content(driver):
    # 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')    
    # 2. 본문 내용 가져오기
    try:  			#여러 태그중 첫번째([0]) 태그를 선택  
        content = soup.select('div.C4VMK > span')[0].text 
        			#첫 게시글 본문 내용이 <div class="C4VMK"> 임을 알 수 있다.
                                #태그명이 div, class명이 C4VMK인 태그 아래에 있는 span 태그를 모두 선택.
    except:
        content = ' ' 
    # 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content) # content 변수의 본문 내용 중 #으로 시작하며, #뒤에 연속된 문자(공백이나 #, \ 기호가 아닌 경우)를 모두 찾아 tags 변수에 저장
    # 4. 작성 일자 가져오기
    try:
        date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10] #앞에서부터 10자리 글자
    except:
        date = ''
    # 5. 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1] 
    except:
        like = 0
    # 6. 위치 정보 가져오기
    try:
        place = soup.select('div.JF9hh')[0].text
    except:
        place = ''
    data = [content, date, like, place, tags]
    return data 


In [192]:
#함수 정의: 첫 게시물 클릭 후 다음 게시물 클릭
def move_next(driver):
  right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow') 
  right.click()
  time.sleep(3)

In [193]:
# install chromium, its driver, and selenium
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [195]:
driver = webdriver.Chrome(options=options)
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(3)

email=''
input_id=driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

passwrod=''
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

word = input("비")
word = str(word)
url = insta_searching(word)

driver.get(url)
time.sleep(3)

#첫 게시물 클릭
select_first(driver)

results= []

target = 10000
for i in range(target):

    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)

results[:2]

KeyboardInterrupt: ignored

In [ ]:
word = input("#")

### IndexError: list index out of range
### 참고: [링크 텍스트](https://korbillgates.tistory.com/91)

In [ ]:
#크롤링 시작
# driver.get(url)을 통해 검색 페이지 접속하고, target 변수에 크롤링할 게시글의 수를 바인딩

driver=